In [1]:
import alpaca_trade_api as tradeapi
import time
import threading
import csv
import pandas as pd

# Alpaca API credentials
API_KEY = 'PKL0QPAXI2XEQN42OD2S'
API_SECRET = 'nkqziIUvRn7rkEihNRVpw9IcNYftpDxVHPRQnyS3'
BASE_URL = 'https://paper-api.alpaca.markets'

# Initialize API
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL)

# News data storage
all_news = []

# Define CSV file name
csv_file = 'financial_news.csv'

# List of tickers
tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "FB", "INTC", "TSLA", "NVDA", "ORCL", "IBM",
    "JPM", "BAC", "WFC", "C", "GS", "AXP", "MS", "BRK.B",
    "JNJ", "PFE", "UNH", "MRK", "ABBV", "TMO", "ABT",
    "PG", "KO", "PEP", "NKE", "MO",
    "GE", "MMM", "BA", "HON", "CAT",
    "XOM", "CVX", "BP", "TOT",
    "NEP", "DUK", "SO",
    "T", "VZ",
    "DIS", "MCD", "SBUX",
    "WMT", "HD"
]

In [2]:
# Function to fetch news for a ticker
def fetch_news(ticker):
    global all_news
    try:
        news = api.get_news(symbol=ticker, limit=200)  # Adjust the limit as needed
        
        for article in news:
            all_news.append({
                'headline': article.headline,
                'summary': article.summary,
                'created_at': article.created_at,
            })
    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")

# Function to process requests in batches
def process_requests():
    while tickers:
        threads = []
        for _ in range(200):  # 200 parallel threads, respecting the rate limit
            if tickers:
                ticker = tickers.pop(0)
                thread = threading.Thread(target=fetch_news, args=(ticker,))
                threads.append(thread)
                thread.start()

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        # Wait for 60 seconds before the next batch
        time.sleep(60)

# Write data to CSV
def write_to_csv():
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=all_news[0].keys())
        writer.writeheader()
        for news_item in all_news:
            writer.writerow(news_item)

    print(f'Data saved to {csv_file}')

In [3]:
import re

# Keep digits and symbols that can be significant in financial analysis. 
# E.g. digits could represent financial figures, and symbols like '$' might indicate currency amounts. 
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)  
    # Remove newlines and tabs
    text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ') 
#     # Keep words, digits, $ and .
#     text = re.sub(r'[^\w\d$.]', ' ', text)  
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)  
    return text.strip()

In [4]:
'''
Testing clean_data()
'''
# Example DataFrame
data = pd.DataFrame({
    'article': ["Stocks rallied, with AAPL reaching $150.50. More at http://example.com", 
                "Q1 profits fell to $1.5 billion, down from $2 billion. Visit http://finance.com for more.", 
                "Tech stocks, e.g., MSFT, AMZN, and GOOG, showed mixed results today."]
})

data['cleaned_article'] = data['article'].apply(clean_text)
print(data)

                                             article  \
0  Stocks rallied, with AAPL reaching $150.50. Mo...   
1  Q1 profits fell to $1.5 billion, down from $2 ...   
2  Tech stocks, e.g., MSFT, AMZN, and GOOG, showe...   

                                     cleaned_article  
0  Stocks rallied, with AAPL reaching $150.50. Mo...  
1  Q1 profits fell to $1.5 billion, down from $2 ...  
2  Tech stocks, e.g., MSFT, AMZN, and GOOG, showe...  


In [5]:
from datetime import datetime
def extract_date(date_str):
    try:
        # Parse the string to a datetime object
        dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S+00:00')
        # Format the datetime object to keep only the date part
        return dt.strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None

In [6]:
'''
Testing extract_date()
'''
date_string = "2024-01-25 13:37:00+00:00"
extracted_date = extract_date(date_string)
print(extracted_date)

2024-01-25


In [7]:
# # Gather data and write to csv file
# process_requests()
# write_to_csv()

In [29]:
# from transformers import pipeline

# classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# def get_sentiment(text):
#     sentiment = classifier(text)[0]  # Access the first element of the result

#     if sentiment['label'] == 'POSITIVE':
#         return f"{sentiment['score']:.2f}"  # Corrected format string

#     return f"-{sentiment['score']:.2f}"  # Corrected format string

df = pd.read_csv(csv_file)

# Data cleaning
df = df.dropna().reset_index(drop=True)
df['headline'] = df['headline'].apply(clean_text)
df['summary'] = df['summary'].apply(clean_text)
df['created_at'] = df['created_at'].apply(extract_date)
df['headline_summary'] = df['headline'] + " " + df['summary']

df

,headline,summary,created_at,headline_summary
0,Meta Blocks Adults From Messaging Teens They D...,Meta Platforms has unveiled stricter messaging...,2024-01-25,Meta Blocks Adults From Messaging Teens They D...
1,"After 130% Surge, AMD Faces Uncertain Road Ahe...",AMD&#39;s stock surged in 2023 due to high dem...,2024-01-23,"After 130% Surge, AMD Faces Uncertain Road Ahe..."
2,Legendary Investor Jim Rogers Predicts Market ...,Rogers has identified indicators of an impendi...,2023-12-29,Legendary Investor Jim Rogers Predicts Market ...
3,Mark Zuckerberg's Meta Said To Be Worried Abou...,This story was first published on the Benzinga...,2023-12-21,Mark Zuckerberg's Meta Said To Be Worried Abou...
4,"Now, Facebook And Messenger Also Get End-To-En...",Meta&#39;s new encryption feature implies that...,2023-12-07,"Now, Facebook And Messenger Also Get End-To-En..."
...,...,...,...,...
6797,"Musk, Bezos, Buffett Get Richer, Rest Of World...",A new report highlights the inequality in weal...,2024-01-17,"Musk, Bezos, Buffett Get Richer, Rest Of World..."
6798,Tesla Halts Model 3 Deliveries in Australia Ov...,Tesla has reportedly halted deliveries of its ...,2024-01-17,Tesla Halts Model 3 Deliveries in Australia Ov...
6799,What's Going On With Tesla Stock?,Tesla shares appear to be facing selling press...,2024-01-17,What's Going On With Tesla Stock? Tesla shares...
6800,Tesla Stock Falls Over 1% Premarket: What's Go...,The stock has been on a lean trot since mid-Ju...,2024-01-17,Tesla Stock Falls Over 1% Premarket: What's Go...


In [30]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Tokenisation
df['tokens'] = df['headline_summary'].apply(word_tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\scsar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
# Removing Stop Words
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df['filtered_tokens'] = df['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\scsar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['filtered_tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\scsar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
# Keyword analysis

def has_direct_impact(text, keywords):
    return any(keyword.lower() in text.lower() for keyword in keywords)

In [ ]:
# Sentiment Analysis
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import numpy as np

# Load pre-trained model and tokenizer
model_name = "yiyanghkust/finbert-tone"  # Example FinBERT model
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Create a sentiment analysis pipeline
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Function to apply FinBERT to a batch of texts
def apply_finbert_to_batch(texts):
    results = finbert(texts)
    labels = [result['label'] for result in results]
    scores = [result['score'] for result in results]
    return labels, scores

# Batch processing function
def process_in_batches(dataframe, column_name, batch_size=20):
    num_batches = int(np.ceil(len(dataframe) / batch_size))
    labels, scores = [], []

    for i in range(num_batches):
        batch_texts = dataframe[column_name].iloc[i * batch_size:(i + 1) * batch_size].tolist()
        batch_labels, batch_scores = apply_finbert_to_batch(batch_texts)
        labels.extend(batch_labels)
        scores.extend(batch_scores)

    return labels, scores

df['sentiment_label'], df['sentiment_score'] = process_in_batches(df, 'headline_summary', batch_size=20)

In [ ]:
import random

test_num = random.randint(1, 6000)
print(df.iloc[test_num][0])
print(df.iloc[test_num][1])
print(df.iloc[test_num][-1])

df